# Python Example for STM32F030/51 MCU
![stm32 remcu](img/title_python.png)

In [ ]:
import ctypes
import sys

[STM32F051_StdPeriph_Lib.py](remcu_include/STM32F051_StdPeriph_Lib.py) is a Python wrapper that containing all necessary constants and structures which makes it possible to use the Standard Peripheral Library function just like with C and C++ code.

In [ ]:
from remcu_include.STM32F051_StdPeriph_Lib import *

Loading the REMCU library

In [ ]:
if sys.platform.startswith('win32'): #Windows
    remcu = ctypes.WinDLL("remcu.dll")
elif sys.platform.startswith('cygwin'): #Windows/Cygwin
    remcu = ctypes.WinDLL("remcu.dll")
elif sys.platform.startswith('linux'):  #Linux OS
    remcu = ctypes.CDLL("./libremcu.so")
elif sys.platform.startswith('darwin'): #MacOS
    remcu = ctypes.CDLL("./libremcu.dylib")

Connection to a debug server. The [REMCU](https://remotemcu.github.io/) library is able to work with [OpenOCD](https://github.com/ilg-archived/openocd/releases/tag/v0.10.0-12-20190422) or [GDB server](https://atollic.com/resources/download/). There is using the ***remcu_connect2OpenOCD***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function for connecting to the OpenOcd server. The functions have the same set of parameters. The first parameter is an IP address of the debug server. The second one is a port of the debug server. For instance, OpenOCD server print the port in log messages. The third parameter is a connection timeout. Upon a successful connection, the function returns the “true” value, otherwise, it returns “false”.

> Detailed description of other REMCU function you see in [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)

Before that, plug stm32 board with debugger to PC
![Image](img/connection.png)
run openocd utility in command line:
```bash
openocd -f interface/stlink-v2.cfg -f target/stm32f0x.cfg
```
Result:
![Image](img/openocd.png)

In [ ]:
debug_server_ip = "127.0.0.1"
default_openocd_port = 6666
default_openocd_gdb_port = 3333
default_jlink_port = 2331
default_stlink_port = 61234
#custom_port = ...
timeout_sec = 3 # It can not be negative

In [ ]:
success = remcu.remcu_connect2OpenOCD(debug_server_ip, default_openocd_port, timeout_sec)

If you use GDB server (either OpenOCD or ST-Link GDB server or Jlink GDB server etc.), you uncomment code below and run it:
<details>
  <summary> Jlink windows. Where the port is written <b> (click here)</b></summary>
 
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink.png" >
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink2.png" >
</details>  
<details>
  <summary> ST link GDB server windows. Where the port is written<b> (click here)</b></summary>

<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/run-stling-gdb.png">
</details>

In [ ]:
# For OpenOCD
#success = remcu.remcu_connect2GDB(debug_server_ip, default_openocd_gdb_port, timeout_sec)
# For Jlink
#success = remcu.remcu_connect2GDB(debug_server_ip, default_jlink_port, timeout_sec)
# For ST link GDB server
#success = remcu.remcu_connect2GDB(debug_server_ip, default_stlink_port, timeout_sec)

In [ ]:
if success == 0:
    print("Server error. Possible solutions: ")
    print("1. Check connection to debug server")
    print("2. Check debug server running")
    print("3. Run the script using Python2")
    exit()

If connection were failed and the function returned false. See [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page) 

If connection were success... Jupyter Output:
![Image](img/success_connection.png)

Reset the MCU (see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)):

In [ ]:
remcu.remcu_resetRemoteUnit(__HALT);

At this stage the microcontroller clock setting is already configured, this is done through SystemInit() function which is called from startup file (startup_stm32f0xx.s) before to branch to application main. To see the default setting of SystemInit() function, refer to remcu_include/startup_stm32f0xx.c file

In [ ]:
remcu.SystemInit();

### GPIO Example
>The code below have been taken from **GPIO_IOToggle** ([main.c](STM32F0xx_StdPeriph_Examples/GPIO/GPIO_IOToggle/main.c)) example of [Standard Peripheral Library(SPL) for STM32F0x line](https://www.st.com/en/embedded-software/stsw-stm32048.html)

### [STM32F030 board](https://stm32-base.org/boards/STM32F030F4P6-STM32F030-DEMO-BOARD-V1.1.html#User-LED)
![stm32f30 gpio](img/30_gpio.png)

LED GPIO Periph clock enable:

In [ ]:
remcu.RCC_AHBPeriphClockCmd(RCC_AHBPeriph_GPIOC, ENABLE);

Configure PA4 in output pushpull mode:

In [ ]:
GPIO_InitStructure = GPIO_InitTypeDef()
GPIO_InitStructure.GPIO_Pin = GPIO_Pin_4;
GPIO_InitStructure.GPIO_Mode = GPIO_Mode_OUT;
GPIO_InitStructure.GPIO_OType = GPIO_OType_PP;
GPIO_InitStructure.GPIO_Speed = GPIO_Speed_2MHz;
GPIO_InitStructure.GPIO_PuPd = GPIO_PuPd_NOPULL;
remcu.GPIO_Init(GPIOA, GPIO_InitStructure.ref());

Flash led on:

In [ ]:
remcu.GPIO_SetBits(GPIOA, GPIO_Pin_4);

Flash led off:

In [ ]:
remcu.GPIO_ResetBits(GPIOA, GPIO_Pin_4);

### [STM32F051 board](https://stm32-base.org/boards/STM32F051C8T6-LC-Technology#User-LED)
![stm32f30 gpio](img/51_gpio.png)

LED GPIO Periph clock enable:

In [ ]:
remcu.RCC_AHBPeriphClockCmd(RCC_AHBPeriph_GPIOC, ENABLE);

Configure PC13 in output pushpull mode:

In [ ]:
GPIO_InitStructure = GPIO_InitTypeDef()
GPIO_InitStructure.GPIO_Pin = GPIO_Pin_13;
GPIO_InitStructure.GPIO_Mode = GPIO_Mode_OUT;
GPIO_InitStructure.GPIO_OType = GPIO_OType_PP;
GPIO_InitStructure.GPIO_Speed = GPIO_Speed_2MHz;
GPIO_InitStructure.GPIO_PuPd = GPIO_PuPd_NOPULL;
remcu.GPIO_Init(GPIOC, GPIO_InitStructure.ref());

Flash led on:

In [ ]:
remcu.GPIO_SetBits(GPIOC,GPIO_Pin_13);

Flash led off:

In [ ]:
remcu.GPIO_ResetBits(GPIOC, GPIO_Pin_13);

If it didn’t happen, you can use the utility functions to detect errors...

### Utility Functions

The ***remcu_getErrorCount***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function returns the number of errors that occurred after connection.

In [ ]:
remcu.remcu_getErrorCount()

If no error occurred, result **0**, otherwise see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page)

The ***remcu_isConnected***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function indicates status of the current debug server connection.

In [ ]:
remcu.remcu_isConnected()

Result: connected ***1*** , disconnected ***0***

Some of debug adapters can have hardware problems. The ***remcu_debuggerTest***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function enables to detect such adapters. The function returns a null pointer if the test is successful.

In [ ]:
remcu.remcu_debuggerTest()

If no error occurred, result **0**, otherwise see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page)

>If bugs are not discovered and the chip don't responds to driver function call or the result of work was not what you expected, see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.github.io/troubleshooting-page)

### ADC example
![adc stm32f0x](img/adc_scheme.png)
>The code below is based on **ADC_BasicExample** ([main.c](STM32F0xx_StdPeriph_Examples/ADC/ADC_BasicExample/main.c)) example of [Standard Peripheral Library(SPL) for STM32F0x line](https://www.st.com/en/embedded-software/stsw-stm32048.html)

In [ ]:
#GPIOC Periph clock enable:
remcu.RCC_APB2PeriphClockCmd(RCC_APB2Periph_ADC1, ENABLE)

#enable ADC system clock:
remcu.RCC_AHBPeriphClockCmd(RCC_AHBPeriph_GPIOA, ENABLE)

GPIO_InitStructure = GPIO_InitTypeDef()

#/* Configure PA.05 (ADC Channel5) as analog input -------------------------*/
GPIO_InitStructure.GPIO_Pin = GPIO_Pin_5
GPIO_InitStructure.GPIO_PuPd = GPIO_PuPd_NOPULL
GPIO_InitStructure.GPIO_Mode = GPIO_Mode_AN
remcu.GPIO_Init(GPIOA, GPIO_InitStructure.ref())

#define ADC base config:

#/* ADCs DeInit */  
remcu.ADC_DeInit(ADC1);

ADC_InitStructure = ADC_InitTypeDef()

#/* Initialize ADC structure */
remcu.ADC_StructInit(ADC_InitStructure.ref())

#/* Configure the ADC1 in continuous mode with a resolution equal to 12 bits  */
ADC_InitStructure.ADC_Resolution = ADC_Resolution_12b
ADC_InitStructure.ADC_ContinuousConvMode = ENABLE
ADC_InitStructure.ADC_ExternalTrigConvEdge = ADC_ExternalTrigConvEdge_None
ADC_InitStructure.ADC_DataAlign = ADC_DataAlign_Right
ADC_InitStructure.ADC_ScanDirection = ADC_ScanDirection_Upward
remcu.ADC_Init(ADC1,  ADC_InitStructure.ref());

#/* ADC1 regular channel5 configuration */ 
remcu.ADC_ChannelConfig(ADC1, ADC_Channel_5, ADC_SampleTime_239_5Cycles)

#/* ADC Calibration */
remcu.ADC_GetCalibrationFactor(ADC1)

#/* Enable ADC1 */
remcu.ADC_Cmd(ADC1, ENABLE)

#/* Check the end of ADC1 reset calibration register */
while remcu.ADC_GetFlagStatus(ADC1, ADC_FLAG_ADRDY) == 0: pass

#/* Start ADC1 Software Conversion */ 
remcu.ADC_StartOfConversion(ADC1)

Start conversion and read ADC value:

In [ ]:
def ADC_value():
    #// start conversion (will be endless as we are in continuous mode)
    while remcu.ADC_GetFlagStatus(ADC1, ADC_FLAG_EOC) == RESET: pass
    return remcu.ADC_GetConversionValue(ADC1)

ADC_value()

Plug a potentiometer into the PA5 pin and run the code below.
Rotate the shaft and you will get the voltage versus time chart:
>First install [matplotlib](https://github.com/matplotlib/jupyter-matplotlib) or make sure you have it

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

print("Result: ")

fig = plt.figure()
ax = fig.add_subplot(111)
plt.ion()

fig.show()
fig.canvas.draw()

ADC = []
for i in range(0,150):
    ADC.append(ADC_value())
    ax.clear()
    ax.plot(ADC)
    fig.canvas.draw()

<details>
  <summary>Result<b>(click to show)</b></summary>

![Image](img/f103_adc.gif)
</details>

Finalizing work with REMCU Library:

In [ ]:
remcu.remcu_disconnect()

### Extra Utility Function

The ***ReMCU_getVersion***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.github.io/api-v1-0)) function returns a pointer to a char array containing information about the REMCU lib versions

In [ ]:
print(ctypes.c_char_p(remcu.remcu_getVersion()).value)